# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Léandre Guertin (1841782)
    - Christella Cidolit (1788096)




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [1]:
import numpy as np
import math
import copy
from collections import deque

IMPOSSIBLE_SCORE = 999999

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        # TODO
        s.rock = self.rock
        return s

    def put_rock(self, rock_pos):
        # TODO
        s = State(self.pos)
        s.prev = self
        
        s.c = self.c
        s.d = self.d
        s.nb_moves = self.nb_moves
        
        s.rock = rock_pos
        
        return s
            
    def score_state(self, rh, is_max=True):
        
        dist_red_exit = (4 - self.pos[0]) 
        best_score_unblocking_red = IMPOSSIBLE_SCORE if is_max else 0


        for car in range(1, rh.nbcars):
            if self.is_car_blocked_by_car(rh, 0, car) == 1:
                if is_max:
                    best_score_unblocking_red = min(best_score_unblocking_red, self.nb_cars_blocking(rh, car, (rh.move_on[0], self.pos[0] + rh.length[0]), 1, is_player_turn=True))
                else:
                    best_score_unblocking_red = max(best_score_unblocking_red, self.nb_cars_blocking(rh, car, (rh.move_on[0], self.pos[0] + rh.length[0]), 1, is_player_turn=False))

        if best_score_unblocking_red == IMPOSSIBLE_SCORE:
            best_score_unblocking_red = 0

        self.score += - 100 * dist_red_exit - best_score_unblocking_red

        # Prevent the cars from going back and forth
        if self.prev and self.c == self.prev.c and self.d != self.prev.d:
            self.score -= 100

    def nb_cars_blocking(self, rh, car_selected, collision_pos, depth, is_player_turn=True):
        nb_cars_blocked = 999999 if is_player_turn else -999999

        # Only have DFS look in a depth of 5 maximum to prevent infinite loops
        if depth >= 4:
            return 0

        # print("In car : ", rh.color[car_selected], " Collision : ", collision_pos)

        if rh.horiz[car_selected]:
            mvts_left = rh.length[car_selected] - (collision_pos[1] - self.pos[car_selected])
            mvts_right = rh.length[car_selected] - mvts_left + 1

            # Move left
            if self.pos[car_selected] - mvts_left >= 0:
                # print("Car :", rh.color[car_selected], " mtvs_left: ", mvts_left)
                if is_player_turn:
                    nb_cars_blocked = min(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, -mvts_left, depth, is_player_turn))
                else:
                    nb_cars_blocked = max(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, -mvts_left, depth, is_player_turn))
                nb_cars_blocked += mvts_left
            
            # Move right
            if self.pos[car_selected] + rh.length[car_selected] - 1 + mvts_right <= 5:
                # print("Car :", rh.color[car_selected], " mtvs_right: ", mvts_right)
                if is_player_turn:
                    nb_cars_blocked = min(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, mvts_right, depth, is_player_turn))
                else:
                    nb_cars_blocked = max(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, mvts_right, depth, is_player_turn))
                nb_cars_blocked += mvts_right

        else:
            mvts_up = rh.length[car_selected] - (collision_pos[0] - self.pos[car_selected])
            mvts_down = rh.length[car_selected] - mvts_up + 1

            # Move up
            if self.pos[car_selected] - mvts_up >= 0:
                # print("Car :", rh.color[car_selected], " mtvs_up: ", mvts_up)
                if is_player_turn:
                    nb_cars_blocked = min(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, -mvts_up, depth, is_player_turn))
                else:
                    nb_cars_blocked = max(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, -mvts_up, depth, is_player_turn))
                nb_cars_blocked += mvts_up

            # Move down
            if self.pos[car_selected] + rh.length[car_selected] - 1 + mvts_down <= 5:
                # print("Car :", rh.color[car_selected], " mvts_down: ", mvts_down)
                if is_player_turn:
                    nb_cars_blocked = min(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, mvts_down, depth, is_player_turn))
                else:
                    nb_cars_blocked = max(nb_cars_blocked, self.calculate_blocking_and_blocked_cars(rh, car_selected, mvts_down, depth, is_player_turn))
                nb_cars_blocked += mvts_down
                
        return nb_cars_blocked

    def calculate_blocking_and_blocked_cars(self, rh, car_selected, mvts, depth, is_player_turn=True):
        nb_cars_blocking = 0
        nb_cars_blocked = 0

        # Calculate overlapping cars
        overlapping_cars = self.find_overlapping_cars_by_move(rh, car_selected, mvts)

        nb_cars_blocking += len(overlapping_cars)

        for car in overlapping_cars:
            # print("Overlapping car : ", rh.color[car])   
            nb_cars_blocking += self.nb_cars_blocking(rh, car, self.find_overlapping_collision(rh, car_selected, mvts, car), depth + 1, is_player_turn)
            # print("Back from Overlapping car : ", rh.color[car])   

        # Calculate cars blocking cars blocking exit
        p_cars = self.find_perpendicular_cars(rh, car_selected, mvts)

        nb_cars_blocked += len(p_cars)

        for p_car in p_cars:
            p_p_cars = self.find_perpendicular_cars(rh, p_car, 0)

            # if the p_p_cars (vertical) are blocking the exit, calculate what it would take them to exit
            for p_p_car in p_p_cars:
                if p_p_car == car_selected:
                    nb_cars_blocked += 1
                    continue

                if rh.move_on[p_p_car] >= self.pos[0] + rh.length[0] and self.pos[p_p_car] <= rh.move_on[0] and self.pos[p_p_car] + rh.length[p_p_car] > rh.move_on[0]:
                    # print("p_p_car : ", rh.color[p_p_car])   
                    nb_cars_blocked += self.nb_cars_blocking(rh, p_p_car, (rh.move_on[0], rh.move_on[p_p_car]), depth + 2, is_player_turn)
                    # print("Back from p_p_car : ", rh.color[p_p_car])

        if not is_player_turn and self.is_rock_blocking(rh, car_selected):
            nb_cars_blocked += (5-depth)

        return nb_cars_blocking + nb_cars_blocked


    def is_car_blocked_by_car(self, rh, subject, target): # Return -1 if behind target, 0 if not blocking and 1 if in front of target
        if rh.horiz[subject] != rh.horiz[target]: # Subject vertical and target horizontal, or subject horizontal and target vertical
            if self.pos[target] <= rh.move_on[subject] and self.pos[target] + rh.length[target] > rh.move_on[subject]: # Check if crosses the rows in front 
                if rh.move_on[target] == self.pos[subject] - 1: # Check if directly behind
                    return -1
                if rh.move_on[target] == self.pos[subject] + rh.length[subject]: # Check if directly in front # TODO: should be <= for the jaune?
                    return 1

        else: # Subject vertical and target vertical or subject horizontal and target horizontal
            if rh.move_on[subject] == rh.move_on[target]: # Check if on same row or col
                if self.pos[target] + rh.length[target] == self.pos[subject]: # Directly above
                    return -1
                if self.pos[target] == self.pos[subject] + rh.length[subject]: # Directly under
                    return 1

        return 0

    def find_overlapping_cars_by_move(self, rh, subject, offset):
        overlapped_cars = set()

        for car in range(0, rh.nbcars):
            if car == subject:
                continue

            if rh.horiz[subject] == rh.horiz[car]:
                if rh.move_on[subject] == rh.move_on[car]:
                    if ((self.pos[car] <= self.pos[subject] + offset and self.pos[car] + rh.length[car] > self.pos[subject] + offset)
                        or (self.pos[car] < self.pos[subject] + rh.length[subject] + offset and self.pos[car] + rh.length[car] >= self.pos[subject] + rh.length[subject] + offset)):
                        overlapped_cars.add(car)
            else:
                if (self.pos[car] <= rh.move_on[subject] and self.pos[car] + rh.length[car] > rh.move_on[subject]
                    and self.pos[subject] + offset <= rh.move_on[car] and self.pos[subject] + offset + rh.length[subject] > rh.move_on[car]):
                    overlapped_cars.add(car)

        return overlapped_cars

    def find_overlapping_collision(self, rh, subject, offset, target):
        if rh.horiz[subject] and not rh.horiz[target]:
            return (rh.move_on[subject], rh.move_on[target])
        elif not rh.horiz[subject] and rh.horiz[target]:
            return (rh.move_on[target], rh.move_on[subject])
        elif rh.horiz[subject] and rh.horiz[target]:
            if offset > 0:
                return (rh.move_on[subject], self.pos[target])
            else:
                return (rh.move_on[subject], self.pos[target] + rh.length[target] - 1)
        else:
            if offset > 0:
                return (self.pos[target], rh.move_on[subject])
            else:
                return (self.pos[target] + rh.length[target] - 1, rh.move_on[subject])

    def find_perpendicular_cars(self, rh, subject, offset):
        perpendicular_cars = set()

        for car in range(0, rh.nbcars):
            if car == subject:
                continue

            if rh.horiz[car] != rh.horiz[subject]:
                if rh.move_on[car] >= self.pos[subject] + offset and rh.move_on[car] < self.pos[subject] + rh.length[subject] + offset:
                    perpendicular_cars.add(car)


        return perpendicular_cars

    def is_rock_blocking(self, rh, car_selected):
        if self.rock:
            if rh.horiz[car_selected] and rh.move_on[car_selected] == self.rock[0]:
                if (self.pos[car_selected] - 1) == self.rock[1] or (self.pos[car_selected] + rh.length[car_selected]) == self.rock[1]:
                    return True
            elif not rh.horiz[car_selected] and rh.move_on[car_selected] == self.rock[1]:
                if (self.pos[car_selected] - 1) == self.rock[0] or (self.pos[car_selected] + rh.length[car_selected]) == self.rock[0]:
                    return True
        return False
        

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [2]:
import numpy as np

class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        # TODO
        if state.rock:
            self.free_pos[state.rock] = False
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        
        for row in range(0,6):
            if row == 2:
                continue
            
            for col in range(0,6):
                if self.free_pos[row][col] == True and (state.rock is None or (state.rock[0] is not row and state.rock[1] is not col)):
                    new_states.append(state.put_rock((row, col)))
        
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False True False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [3]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [4]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    # Test put rock at (3,4)
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [5]:
import numpy as np
import enum
import random
import math

class Algorithm(enum.Enum):
    MINIMAX_SINGLE = 1
    MINIMAX_MULTI = 2
    PRUNING = 3
    EXPECTIMAX = 4

class ExpectimaxProbability(enum.Enum):
    RANDOM = 1
    OPTIMISTIC = 2
    PESSIMISTIC = 3


class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.nb_moves_tot = 0
        self.nb_state_searched = 0
        self.expectimax_probability = ExpectimaxProbability.RANDOM

    #  Un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant
    def minimax_1(self, current_depth, current_state): 
        #TODO
        self.nb_state_searched += 1

        best_score = None
        possible_states = self.rushhour.possible_moves(current_state)
        
        if current_depth == self.search_depth:
            current_state.score_state(self.rushhour)
            return current_state.score
        
        for state in possible_states:
            score = self.minimax_1(current_depth + 1, state)        

            if best_score is None:
                best_score = score

            best_score = max(best_score, score)

        return best_score
        
    
    def minimax_2(self, current_depth, current_state, is_max, is_rock_turn): 
        #TODO
        self.nb_state_searched += 1

        if current_depth == self.search_depth:
            current_state.score_state(self.rushhour, is_rock_turn)
            return current_state.score

        best_score = None

        if is_max:
            possible_states = self.rushhour.possible_moves(current_state)
        else:
            possible_states = self.rushhour.possible_rock_moves(current_state)
        
        for state in possible_states:
            score = self.minimax_2(current_depth + 1, state, not is_max, is_rock_turn)        

            if best_score is None:
                best_score = score

            if is_max:
                best_score = max(best_score, score)
            else:
                best_score = min(best_score, score)

        return best_score

    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta, is_rock_turn):

        self.nb_state_searched += 1
        best_score = None

        if current_depth == self.search_depth:
            current_state.score_state(self.rushhour, is_rock_turn)
            return current_state.score

        if is_max:
            possible_states = self.rushhour.possible_moves(current_state)
        else:
            possible_states = self.rushhour.possible_rock_moves(current_state)
        
        for state in possible_states:
            score = self.minimax_pruning(current_depth + 1, state, not is_max, alpha, beta, is_rock_turn)        

            if best_score is None:
                best_score = score

            if is_max:
                best_score = max(best_score, score)
                if score >= beta:
                    return best_score
                
                alpha = max(alpha, score)
            else:
                best_score = min(best_score, score)
                
                if score <= alpha:
                    return best_score
                
                beta = min(beta, score)
                
        return best_score

    def expectimax(self, current_depth, current_state, is_max, is_rock_turn):
        #TODO
        self.nb_state_searched += 1

        if current_depth == self.search_depth:
            current_state.score_state(self.rushhour, is_rock_turn)
            return current_state.score

        best_score = None

        if is_max:
            possible_states = self.rushhour.possible_moves(current_state)

            for state in possible_states:
                score = self.expectimax(current_depth + 1, state, not is_max, is_rock_turn)        

                if best_score is None:
                    best_score = score

                if is_max:
                    best_score = max(best_score, score)
                else:
                    best_score = min(best_score, score)

            return best_score
        else:
            possible_states = self.rushhour.possible_rock_moves(current_state)

            scores = []
            for state in possible_states:
                scores.append(self.expectimax(current_depth + 1, state, not is_max, is_rock_turn)) 

            p = 1
            scores_len = len(scores)
            p = p / scores_len
            p_arr = []

            if self.expectimax_probability == ExpectimaxProbability.RANDOM:
                for idx in range(0, scores_len):
                    p_arr.append(p)

            else:
                mid = math.floor(scores_len / 2)
                
                for idx in range(0, scores_len):
                    p_arr.append(p + (idx - mid) * p / scores_len)

                if sum(p_arr) != 1.0:
                    p_left = 1.0 - sum(p_arr)

                    for idx in range(0, scores_len):
                        p_arr[idx] = p_arr[idx] + p_left / scores_len

                if self.expectimax_probability == ExpectimaxProbability.OPTIMISTIC: # probability higher for lower scores
                    scores.sort(reverse=True)

                elif self.expectimax_probability == ExpectimaxProbability.PESSIMISTIC: # probability higher for higher scores
                    scores.sort()

            scores = [a*b for a,b in zip(scores,p_arr)]

            return sum(scores)

        return best_score

    # Trouve et exécute le meilleur coup pour une partie à un joueur (apelle version de mininmax)
    def decide_best_move_1(self):
        #TODO
        
        best_move = None

        possible_states = self.rushhour.possible_moves(self.state)
        for state in possible_states:
            state.score = self.minimax_1(1, state)

            if best_move is None:
                best_move = state

            best_move = max(best_move, state)
        
        return best_move

    def decide_best_move_2(self, is_max, is_rock_turn):
        #TODO
        best_move = None

        if is_max:
            possible_states = self.rushhour.possible_moves(self.state)
        else:
            possible_states = self.rushhour.possible_rock_moves(self.state)
            
        for state in possible_states:
            state.score = self.minimax_2(1, state, not is_max, is_rock_turn)

            if best_move is None:
                best_move = state

            if is_max:
                best_move = max(best_move, state)
            else:
                best_move = min(best_move, state)

        
        return best_move

    def decide_best_move_pruning(self, is_max, is_rock_turn):
        #TODO
        best_move = None
        alpha = -1000000
        beta = 1000000

        if is_max:
            possible_states = self.rushhour.possible_moves(self.state)
        else:
            possible_states = self.rushhour.possible_rock_moves(self.state)
            
        for state in possible_states:
            state.score = self.minimax_pruning(1, state, not is_max, alpha, beta, is_rock_turn)

            if best_move is None:
                best_move = state

            if is_max:
                best_move = max(best_move, state)
                if state.score >= beta:
                    return best_move
                
                alpha = max(alpha, state.score)
            else:
                best_move = min(best_move, state)
                
                if state.score <= alpha:
                    return best_move
                
                beta = min(beta, state.score)
        
        return best_move

    def decide_best_move_expectimax(self, is_max, is_rock_turn):
        #TODO
        best_move = None

        if is_max:
            possible_states = self.rushhour.possible_moves(self.state)
        else:
            possible_states = self.rushhour.possible_rock_moves(self.state)
            return random.choice(possible_states)
            
        for state in possible_states:
            state.score = self.expectimax(1, state, not is_max, is_rock_turn)

            if best_move is None:
                best_move = state

            if is_max:
                best_move = max(best_move, state)
            else:
                best_move = min(best_move, state)

        
        return best_move

    def solve(self, state, algorithm): # apelle plusieurs fois decide_best_move
        #TODO
        self.nb_moves_tot = 1

        if algorithm == Algorithm.MINIMAX_SINGLE:
            self.state = self.decide_best_move_1()
            while not self.state.success():
                print('final move: ', end='')
                self.print_move(True, self.state)
                # self.rushhour.print_pretty_grid(self.state)

                self.state = self.decide_best_move_1()
                self.nb_moves_tot += 1
                
        elif algorithm == Algorithm.MINIMAX_MULTI:
            is_max = True
            self.state = self.decide_best_move_2(is_max, is_rock_turn=is_max)
            while not self.state.success():

                print('final move: ', end='')
                self.print_move(is_max, self.state)
                # self.rushhour.print_pretty_grid(self.state)
                
                is_max = not is_max
                self.state = self.decide_best_move_2(is_max, is_rock_turn=is_max)
                
                if is_max:
                    self.nb_moves_tot += 1
                
                if self.nb_moves_tot == 80:
                    print('FAIL')
                    break
        
        elif algorithm == Algorithm.PRUNING:
            is_max = True
            self.state = self.decide_best_move_pruning(is_max, is_rock_turn=is_max)
            while not self.state.success():

                print('final move: ', end='')
                self.print_move(is_max, self.state)
                # self.rushhour.print_pretty_grid(self.state)
                
                is_max = not is_max
                self.state = self.decide_best_move_pruning(is_max, is_rock_turn=is_max)
                
                if is_max:
                    self.nb_moves_tot += 1
                
                if self.nb_moves_tot == 80:
                    print('FAIL')
                    break

        else:
            is_max = True
            self.state = self.decide_best_move_expectimax(is_max, is_rock_turn=is_max)
            while not self.state.success():

                print('final move: ', end='')
                self.print_move(is_max, self.state)
                # self.rushhour.print_pretty_grid(self.state)
                
                is_max = not is_max
                self.state = self.decide_best_move_expectimax(is_max, is_rock_turn=is_max)
                
                if is_max:
                    self.nb_moves_tot += 1
                
                if self.nb_moves_tot == 80:
                    print('FAIL')
                    break

            
    def print_move(self, is_max, state):
        #TODO
        # Is_max : player
        # Not is_max : adversary
        
        if is_max:
            print('Voiture {} vers '.format(self.rushhour.color[state.c]), end='')
            
            if self.rushhour.horiz[state.c] and state.d == -1:
                print('la gauche')
            
            if self.rushhour.horiz[state.c] and state.d == 1:
                print('la droite')
            
            if (not self.rushhour.horiz[state.c]) and state.d == -1:
                print('le haut')
            
            if (not self.rushhour.horiz[state.c]) and state.d == 1:
                print('le bas')
        
        else:
            print('Roche dans la case {}-{}'.format(state.rock[0], state.rock[1]))
        
        pass
          


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [6]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Voiture rouge vers la droite
Roche dans la case 3-1


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [7]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.MINIMAX_SINGLE)
print('\nEND - Nb moves: ', algo.nb_moves_tot, '\n')
%time

Init: 
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
===
final move: Voiture rouge vers la droite
final move: Voiture orange vers le haut
final move: Voiture jaune vers la gauche
final move: Voiture vert vers le bas
final move: Voiture vert vers le bas
final move: Voiture vert vers le bas
final move: Voiture rouge vers la droite
final move: Voiture bleu vers le haut

END - Nb moves:  9 

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.58 µs


In [8]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.MINIMAX_SINGLE) 
print('\nEND - Nb moves: ', algo.nb_moves_tot, '\n')
%time

Init: 
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
===
final move: Voiture lime vers la gauche
final move: Voiture emeraude vers la gauche
final move: Voiture vert vers la droite
final move: Voiture mauve vers le haut
final move: Voiture orange vers le haut
final move: Voiture emeraude vers la gauche
final move: Voiture vert vers la droite
final move: Voiture vert vers la droite
final move: Voiture jaune vers le bas
final move: Voiture vert vers la droite
final move: Voiture lime vers la gauche
final move: Voiture lime vers la gauche
final move: Voiture bleu vers le bas
final move: Voiture bleu vers le bas
final move: Voiture rouge vers la droite
final move: Voiture rouge vers la droite
final move: Voiture jaune vers le bas
final move: Voiture jaune vers le bas

END - Nb moves:  19 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall 

In [9]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.MINIMAX_SINGLE)
print('\nEND - Nb moves: ', algo.nb_moves_tot,'\n')
%time

Init: 
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
===
final move: Voiture rouge vers la droite
final move: Voiture 10 vers la gauche
final move: Voiture 3 vers le bas
final move: Voiture 1 vers le bas
final move: Voiture 2 vers la gauche
final move: Voiture 2 vers la gauche
final move: Voiture 2 vers la gauche
final move: Voiture 3 vers le haut
final move: Voiture 3 vers le haut
final move: Voiture rouge vers la droite
final move: Voiture 4 vers le haut
final move: Voiture 4 vers le haut
final move: Voiture rouge vers la droite
final move: Voiture 5 vers le bas
final move: Voiture 5 vers le bas

END - Nb moves:  16 

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 3.34 µs


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [26]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.MINIMAX_MULTI)
print('\nEND\n- Nb moves: ', algo.nb_moves_tot)
print('- Nb states analyzed: ', algo.nb_state_searched,'\n')
%time

Init: 
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
===
final move: Voiture rouge vers la droite
final move: Roche dans la case 0-0
final move: Voiture vert vers le bas
final move: Roche dans la case 1-1
final move: Voiture orange vers le haut
final move: Roche dans la case 0-0
final move: Voiture jaune vers la gauche
final move: Roche dans la case 1-1
final move: Voiture vert vers le bas
final move: Roche dans la case 5-4
final move: Voiture bleu vers le bas
final move: Roche dans la case 0-0
final move: Voiture vert vers le bas
final move: Roche dans la case 1-1
final move: Voiture bleu vers le bas
final move: Roche dans la case 0-0
final move: Voiture rouge vers la droite
final move: Roche dans la case 1-1

END
- Nb moves:  10
- Nb states analyzed:  18559 

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs


In [27]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.MINIMAX_MULTI) 
print('\nEND\n- Nb moves: ', algo.nb_moves_tot)
print('- Nb states analyzed: ', algo.nb_state_searched,'\n')
%time

Init: 
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
===
final move: Voiture lime vers la gauche
final move: Roche dans la case 5-1
final move: Voiture lime vers la gauche
final move: Roche dans la case 0-2
final move: Voiture lime vers la gauche
final move: Roche dans la case 3-1
final move: Voiture bleu vers le bas
final move: Roche dans la case 0-2
final move: Voiture emeraude vers la gauche
final move: Roche dans la case 3-1
final move: Voiture vert vers la droite
final move: Roche dans la case 0-0
final move: Voiture vert vers la droite
final move: Roche dans la case 3-1
final move: Voiture mauve vers le haut
final move: Roche dans la case 0-4
final move: Voiture orange vers le haut
final move: Roche dans la case 5-0
final move: Voiture vert vers la droite
final move: Roche dans la case 0-1
final move: Voiture emeraude vers la gau

In [28]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.MINIMAX_MULTI)
print('\nEND\n- Nb moves: ', algo.nb_moves_tot)
print('- Nb states analyzed: ', algo.nb_state_searched,'\n')
%time

Init: 
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
===
final move: Voiture 10 vers la gauche
final move: Roche dans la case 0-1
final move: Voiture 3 vers le bas
final move: Roche dans la case 4-0
final move: Voiture rouge vers la droite
final move: Roche dans la case 0-1
final move: Voiture 2 vers la gauche
final move: Roche dans la case 4-5
final move: Voiture 2 vers la gauche
final move: Roche dans la case 0-4
final move: Voiture 1 vers le bas
final move: Roche dans la case 4-5
final move: Voiture 2 vers la gauche
final move: Roche dans la case 0-4
final move: Voiture 3 vers le haut
final move: Roche dans la case 1-1
final move: Voiture 3 vers le haut
final move: Roche dans la case 0-5
final move: Voiture rouge vers la droite
final move: Roche dans la case 3-3
final move: Voiture 4 vers le haut
final move: Roche dans la case 0-4
f

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [29]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')
    
algo.solve(s, Algorithm.PRUNING)
print('\nEND\n- Nb moves: ', algo.nb_moves_tot)
print('- Nb states analyzed: ', algo.nb_state_searched,'\n')
%time

Init: 
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
===
final move: Voiture rouge vers la droite
final move: Roche dans la case 0-0
final move: Voiture vert vers le bas
final move: Roche dans la case 1-1
final move: Voiture orange vers le haut
final move: Roche dans la case 0-0
final move: Voiture jaune vers la gauche
final move: Roche dans la case 1-1
final move: Voiture vert vers le bas
final move: Roche dans la case 5-4
final move: Voiture bleu vers le bas
final move: Roche dans la case 0-0
final move: Voiture vert vers le bas
final move: Roche dans la case 1-1
final move: Voiture bleu vers le bas
final move: Roche dans la case 0-0
final move: Voiture rouge vers la droite
final move: Roche dans la case 1-1

END
- Nb moves:  10
- Nb states analyzed:  6332 

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.58 µs


In [30]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.PRUNING)
print('\nEND\n- Nb moves: ', algo.nb_moves_tot)
print('- Nb states analyzed: ', algo.nb_state_searched,'\n')
%time

Init: 
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
===
final move: Voiture lime vers la gauche
final move: Roche dans la case 5-1
final move: Voiture lime vers la gauche
final move: Roche dans la case 0-2
final move: Voiture lime vers la gauche
final move: Roche dans la case 3-1
final move: Voiture bleu vers le bas
final move: Roche dans la case 0-2
final move: Voiture emeraude vers la gauche
final move: Roche dans la case 3-1
final move: Voiture vert vers la droite
final move: Roche dans la case 0-0
final move: Voiture vert vers la droite
final move: Roche dans la case 3-1
final move: Voiture mauve vers le haut
final move: Roche dans la case 0-4
final move: Voiture orange vers le haut
final move: Roche dans la case 5-0
final move: Voiture vert vers la droite
final move: Roche dans la case 0-1
final move: Voiture emeraude vers la gau

In [22]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.PRUNING)
print('\nEND\n- Nb moves: ', algo.nb_moves_tot)
print('- Nb states analyzed: ', algo.nb_state_searched,'\n')
%time

Init: 
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
===
final move: Voiture 10 vers la gauche
final move: Roche dans la case 0-1
final move: Voiture 3 vers le bas
final move: Roche dans la case 4-0
final move: Voiture rouge vers la droite
final move: Roche dans la case 0-1
final move: Voiture 2 vers la gauche
final move: Roche dans la case 4-5
final move: Voiture 2 vers la gauche
final move: Roche dans la case 0-4
final move: Voiture 1 vers le bas
final move: Roche dans la case 4-5
final move: Voiture 2 vers la gauche
final move: Roche dans la case 0-4
final move: Voiture 3 vers le haut
final move: Roche dans la case 1-1
final move: Voiture 3 vers le haut
final move: Roche dans la case 0-5
final move: Voiture rouge vers la droite
final move: Roche dans la case 3-3
final move: Voiture 4 vers le haut
final move: Roche dans la case 0-4
f

#### Comparaison du nombre de noeuds visités avec et sans Élagage Alpha-Beta

|                                  | 9 moves | 14 moves | 16 moves |
|:--------------------------------:|:-------:|:--------:|:--------:|
|  Nombre de states visités  Sans Élagage |  18559  |   11326  |   16998  |
|   Nombre de states visités Avec Élagage |   6332  |   4001   |   6129   |

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [66]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
algo.expectimax_probability = ExpectimaxProbability.RANDOM
# algo.expectimax_probability = ExpectimaxProbability.OPTIMISTIC
# algo.expectimax_probability = ExpectimaxProbability.PESSIMISTIC

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')
    
algo.solve(s, Algorithm.EXPECTIMAX)
print('\nEND - Nb moves: ', algo.nb_moves_tot,'\n')
%time

Init: 
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]
===
final move: Voiture rouge vers la droite
final move: Roche dans la case 1-0
final move: Voiture orange vers le haut
final move: Roche dans la case 3-3
final move: Voiture jaune vers la gauche
final move: Roche dans la case 1-2
final move: Voiture vert vers le bas
final move: Roche dans la case 4-3
final move: Voiture vert vers le bas
final move: Roche dans la case 0-1
final move: Voiture vert vers le bas
final move: Roche dans la case 5-5
final move: Voiture rouge vers la droite
final move: Roche dans la case 1-2
final move: Voiture bleu vers le haut
final move: Roche dans la case 0-4

END - Nb moves:  9 

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.58 µs


In [63]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
algo.expectimax_probability = ExpectimaxProbability.RANDOM
# algo.expectimax_probability = ExpectimaxProbability.OPTIMISTIC
# algo.expectimax_probability = ExpectimaxProbability.PESSIMISTIC

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.EXPECTIMAX) 
print('\nEND - Nb moves: ', algo.nb_moves_tot,'\n')
%time

Init: 
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]
===
final move: Voiture lime vers la gauche
final move: Roche dans la case 1-2
final move: Voiture lime vers la gauche
final move: Roche dans la case 3-1
final move: Voiture emeraude vers la gauche
final move: Roche dans la case 5-4
final move: Voiture lime vers la gauche
final move: Roche dans la case 3-5
final move: Voiture vert vers la droite
final move: Roche dans la case 5-4
final move: Voiture mauve vers le haut
final move: Roche dans la case 4-5
final move: Voiture orange vers le haut
final move: Roche dans la case 3-1
final move: Voiture emeraude vers la gauche
final move: Roche dans la case 1-4
final move: Voiture bleu vers le bas
final move: Roche dans la case 5-3
final move: Voiture vert vers la droite
final move: Roche dans la case 0-4
final move: Voiture bleu vers le bas

In [62]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
algo.expectimax_probability = ExpectimaxProbability.RANDOM
# algo.expectimax_probability = ExpectimaxProbability.OPTIMISTIC
# algo.expectimax_probability = ExpectimaxProbability.PESSIMISTIC

print('Init: ')
algo.rushhour.print_pretty_grid(s)
print('===')

algo.solve(s, Algorithm.EXPECTIMAX)
print('\nEND - Nb moves: ', algo.nb_moves_tot,'\n')
%time

Init: 
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]
===
final move: Voiture 10 vers la gauche
final move: Roche dans la case 3-3
final move: Voiture rouge vers la droite
final move: Roche dans la case 0-1
final move: Voiture 2 vers la gauche
final move: Roche dans la case 3-3
final move: Voiture 2 vers la gauche
final move: Roche dans la case 4-0
final move: Voiture 1 vers le bas
final move: Roche dans la case 5-5
final move: Voiture 2 vers la gauche
final move: Roche dans la case 1-1
final move: Voiture 3 vers le haut
final move: Roche dans la case 5-5
final move: Voiture rouge vers la droite
final move: Roche dans la case 1-2
final move: Voiture 4 vers le haut
final move: Roche dans la case 5-5
final move: Voiture 4 vers le haut
final move: Roche dans la case 3-4
final move: Voiture 5 vers le bas
final move: Roche dans la case 1-1
f

#### Comparaison de la performance de l'AI en fonction des 3 visions: aléatoire, optimiste ou pessimiste

| Nombre de coups | 9 moves |    14 moves   | 16 moves |
|:---------------:|:-------:|:-------------:|:--------:|
|    Aléatoire    |    9    | 14 (ou perte) |    16    |
|    Optimiste    |    9    | 14 (ou perte) |    16    |
|    Pessimiste   |    9    |     perte     |    18    |


**Analyse**  
On remarque qu'utiliser une vision pessimiste de l'AI a tendance à aboutir à de moins bons résultats qu'une vision optimiste ou aléatoire.  
On peut expliquer cela par le fait que l'AI va s'attendre au mouvement de la roche qui lui sera la moins profitable, et jouera alors de manière rpotective, alors que dans d'autres configurations, l'AI va adopter une stratégie plus à son avantge.  
La roche suivant un comportement aléatoire, on a plus ou autant de chance que notre estimation optimiste ou aléatoire corresponde en effet au mouvement que la roche va effectuer.

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 